In [44]:
import requests
import pandas as pd
import csv
import requests
import sqlite3
import openpyxl
from openpyxl.utils.dataframe import dataframe_to_rows

In [45]:
# Set the API endpoint and parameters to import balance sheet and income statement data for AIG and RGA
url = 'https://www.alphavantage.co/query'
function1 = 'BALANCE_SHEET'
function2 = 'INCOME_STATEMENT'
symbols = ['AIG', 'RGA']
apikey = 'P249K0Q2QUE0A029'
params = {
    'function': function1,
    'apikey': apikey,
}

In [46]:
# Loop through each symbol (AIG and RGA) and retrieve the income statement data
dfs = []
for symbol in symbols:
    params['symbol'] = symbol
    response = requests.get(url, params=params)
    data = response.json()
    balance_sheet_data = data['annualReports']
    df = pd.DataFrame.from_dict(balance_sheet_data)
    df['symbol'] = symbol
    dfs.append(df)

In [47]:
# Combine the data for both symbols into a single DataFrame
df_combined = pd.concat(dfs)

In [48]:
# switch the params to function2 (Balance sheet) in order to prepare to import the balance sheet information
params = {
    'function': function2,
    'apikey': apikey,
}

In [49]:
# import the balance sheet information for each of the symbols into a dataframe called dfs2
dfs2 = []
for symbol in symbols:
    params['symbol'] = symbol
    response = requests.get(url, params=params)
    data = response.json()
    income_statement_data = data['annualReports']
    df = pd.DataFrame.from_dict(income_statement_data)
    df['symbol'] = symbol
    dfs2.append(df)

In [50]:
df2_combined = pd.concat(dfs2)

In [51]:
df_merged = pd.merge(df_combined, df2_combined, on=["fiscalDateEnding","symbol"], how='inner')

In [52]:
print(df_merged.shape)

(10, 64)


In [53]:
df_merged.to_csv('RGA_AIG.csv', index=False)

In [54]:
# Load the existing workbook
workbook = openpyxl.load_workbook(filename="data_source.xlsx")

In [55]:
# Get the "Company_data" sheet
worksheet = workbook["Company_data"]

In [56]:
# Export df_merged to the "Company_data" sheet
for r in dataframe_to_rows(df_merged, header=True):
    worksheet.append(r)

In [57]:
# Save the changes to the workbook
workbook.save("data_source.xlsx")